In [ ]:
!pip install missingno

Data Cleaning Functions:
    1. Normalize date to nearest 15T min increment
    2. Fill empty rows with DataTimeStamps where data is missing
    3. Data quality check
    3. Visualize missing data 
    4. Decide on data imputation

In [ ]:
# Function to check data missingness

def check_missing(df):
    # Check percentage of missing values
    # A year should have 35040 rows

    row_count = len(df)
    threshold_row_count = 0.7 * 35040  # 70% of 35040 rows

    if 
    print ()
    

In [59]:
# Function to normalize time to the nearest 15-minute increment
def round_time_to_15min(dt):
    dt += timedelta(minutes=7.5)
    dt -= timedelta(minutes=dt.minute % 15, seconds=dt.second)
    return dt

In [70]:
# Function to fill gaps in data with empty rows

import pandas as pd
import numpy as np

def fill_gaps(df):
    year = df['DateTimeStamp'].dt.year.iloc[0]          # Get the year from the DateTimeStamp column
    start_time = pd.Timestamp(year, 1, 1)              # Set the start time to the beginning of the year
    end_time = pd.Timestamp(year, 12, 31, 23, 59, 59)     # Set the end time to the end of the year
    expected_times = pd.date_range(start=start_time, end=end_time, freq='15T')  # Generate expected times within the year
    
    # Find missing times within the year
    missing_times_within_year = expected_times[~expected_times.isin(df['DateTimeStamp'])]
    
    # Create missing data DataFrame
    missing_data = pd.DataFrame({
        'DateTimeStamp': missing_times_within_year,
        'Occupancy': np.nan,
        'Volume': np.nan,
        'Speed': np.nan
    })
    
    filled_df = pd.concat([df, missing_data]).sort_values(by='DateTimeStamp')    # Concatenate missing_data with original DataFrame
    
    return filled_df

In [ ]:
# Main script to clean the sensor data

input_directory = 'C:/Users/Tarek Zahid/Desktop/xie/Sensor Files/2. transformed sensor files/2018'
output_directory = 'C:/Users/Tarek Zahid/Desktop/xie/Sensor Files/3. cleaned sensor files/2018'

# Read and process each CSV file
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        input_filepath = os.path.join(input_directory, filename)
        df = pd.read_csv(input_filepath, parse_dates=['DateTimeStamp'])

        # Normalize time to the nearest 15-minute increment
        df['DateTimeStamp'] = df['DateTimeStamp'].apply(round_time_to_15min)

        # Fill gaps in data with empty rows
        df = fill_gaps(df)

        # Save the processed data to a new CSV file
        output_filename = os.path.splitext(filename)[0] + '_processed.csv'
        output_filepath = os.path.join(output_directory, output_filename)
        df.to_csv(output_filepath, index=False)

        print(f"Processed file: {output_filepath}")

In [79]:
# Data Quality Check
# Check if the row count of each CSV file is equal to 35040

def check_csv_row_count(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.csv')]
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        row_count = len(df)
        
        # Check if the row count is not equal to 35040
        if row_count != 35040:
            print(f"{file_path}: {row_count} rows")
          

# Set directory path
check_csv_row_count('C:/Users/Tarek Zahid/Desktop/xie/Sensor Files/3. cleaned sensor files/2018')

In [80]:
# Data Quality Check
# Check for duplicates in column 1 (DateTimeStamp)

def check_duplicates_in_column1(directory):
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    
    for file_name in csv_files:
        file_path = os.path.join(directory, file_name)
        df = pd.read_csv(file_path)
        
        # Calculate row count
        row_count = len(df)
        
        # Check for duplicates in the first column
        duplicates = df[df.duplicated(subset=[df.columns[0]], keep=False)]
        
        if not duplicates.empty:
            print(f"Duplicates found in {file_name} (Total Rows: {row_count}):")
            print(duplicates)
        else:
            print(f"No duplicates found in {file_name} (Total Rows: {row_count})")

# Set directory path
check_duplicates_in_column1('C:/Users/Tarek Zahid/Desktop/xie/Sensor Files/3. cleaned sensor files/2018')


No duplicates found in 10.1.267_processed.csv (Total Rows: 35040)
No duplicates found in 10.2.268_processed.csv (Total Rows: 35040)
No duplicates found in 101.1.35_processed.csv (Total Rows: 35040)
No duplicates found in 102.1.40_processed.csv (Total Rows: 35040)
No duplicates found in 102.2.37_processed.csv (Total Rows: 35040)
No duplicates found in 103.1.40_processed.csv (Total Rows: 35040)
No duplicates found in 103.2.40_processed.csv (Total Rows: 35040)
No duplicates found in 109.1.37_processed.csv (Total Rows: 35040)
No duplicates found in 110.1.41_processed.csv (Total Rows: 35040)
No duplicates found in 111.1.42_processed.csv (Total Rows: 35040)
No duplicates found in 112.2.44_processed.csv (Total Rows: 35040)
No duplicates found in 113.2.45_processed.csv (Total Rows: 35040)
No duplicates found in 114.2.44_processed.csv (Total Rows: 35040)
No duplicates found in 115.1.45_processed.csv (Total Rows: 35040)
No duplicates found in 116.2.45_processed.csv (Total Rows: 35040)
No duplica

In [75]:
# Data Quality Check
# Aggregate duplicates by taking the mean of the values and rounding to the nearest integer for Occupancy and Volume and to 2 decimal places for Speed

def check_and_handle_duplicates(directory):
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    for file_name in csv_files:
        file_path = os.path.join(directory, file_name)
        df = pd.read_csv(file_path)
        duplicate_timestamps = df[df.duplicated(subset=['DateTimeStamp'], keep=False)]
        
        # Handle duplicates by aggregating values
        if not duplicate_timestamps.empty:
            # Aggregate duplicates by taking the mean of numeric columns
            aggregated_df = duplicate_timestamps.groupby('DateTimeStamp').mean().reset_index()
            # Round 'Occupancy' and 'Volume' columns to nearest integer
            aggregated_df['Occupancy'] = aggregated_df['Occupancy'].round().astype(int)
            aggregated_df['Volume'] = aggregated_df['Volume'].round().astype(int)
            
            # Drop duplicate rows from original DataFrame
            df.drop_duplicates(subset=['DateTimeStamp'], keep=False, inplace=True)
            
            # Concatenate aggregated DataFrame with original DataFrame
            df = pd.concat([df, aggregated_df], ignore_index=True)
            
            # Save the updated DataFrame back to CSV file
            df.to_csv(file_path, index=False)

            print(f"Duplicates handled in {file_name}")
        else:
            print(f"No duplicates found in {file_name}")

# Example usage:
check_and_handle_duplicates('C:/Users/Tarek Zahid/Desktop/xie/Sensor Files/3. cleaned sensor files/2018')



No duplicates found in 10.1.267_processed.csv
Duplicates handled in 10.2.268_processed.csv
No duplicates found in 101.1.35_processed.csv
Duplicates handled in 102.1.40_processed.csv
Duplicates handled in 102.2.37_processed.csv
Duplicates handled in 103.1.40_processed.csv
Duplicates handled in 103.2.40_processed.csv
Duplicates handled in 109.1.37_processed.csv
Duplicates handled in 110.1.41_processed.csv
Duplicates handled in 111.1.42_processed.csv
No duplicates found in 112.2.44_processed.csv
Duplicates handled in 113.2.45_processed.csv
No duplicates found in 114.2.44_processed.csv
Duplicates handled in 115.1.45_processed.csv
Duplicates handled in 116.2.45_processed.csv
No duplicates found in 117.1.46_processed.csv
No duplicates found in 117.3.43_processed.csv
No duplicates found in 122.2.48_processed.csv
No duplicates found in 123.2.50_processed.csv
Duplicates handled in 124.2.49_processed.csv
Duplicates handled in 125.2.49_processed.csv
Duplicates handled in 126.1.49_processed.csv
No